Setup PyTorch to use best hardware option

In [1]:
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(device)

cuda


Dataset download

In [2]:
import pandas as pd
import swifter

splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
train_df = pd.read_parquet("hf://datasets/jayavibhav/prompt-injection/" + splits["train"])
test_df = pd.read_parquet("hf://datasets/jayavibhav/prompt-injection/" + splits["test"])

In [3]:
test_df.rename(columns={"text":"prompt"}, inplace=True)
train_df.rename(columns={"text":"prompt"}, inplace=True)

In [4]:
test_df.head()

,prompt,label
0,"Task: Given a list of words, identify which wo...",0
1,So I was watching episode 11 of Dark Angel sea...,0
2,"Given the sentence ""Lots of people standing on...",0
3,"In this task, you are given the name of an Ind...",0
4,"Task: Replace all the sentences that use ""i"" w...",0


In [5]:
train_df.head()

,prompt,label
0,My question is: Alani earned $45 for 3 hours o...,0
1,"Q: Yes / no, is the following a verifiable fac...",0
2,Read a code snippet and detect if there are an...,0
3,"As a highly advanced chatbot, it is your duty ...",1
4,Tell me the main idea of this paragraph.,0


Get model and tokenizer from Hugginface

In [7]:
from transformers import AutoTokenizer, AutoModel

model_name = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name, clean_up_tokenization_spaces=False, use_fast=True)
model = AutoModel.from_pretrained(model_name).to(device)

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [8]:
def get_embedding(prompt):
    tokens = tokenizer(prompt, return_tensors='pt', truncation=True, padding=True, max_length=512).to(device)

    with torch.no_grad():
        outputs = model(**tokens)

    last_hidden_states = outputs.last_hidden_state
    embedding_vector = last_hidden_states.mean(dim=1).squeeze().cpu().numpy()

    return embedding_vector

In [9]:
train_df['embedding'] = train_df['prompt'].swifter.apply(get_embedding)

Pandas Apply:   0%|          | 0/261738 [00:00<?, ?it/s]

In [10]:
test_df['embedding'] = test_df['prompt'].swifter.apply(get_embedding)

Pandas Apply:   0%|          | 0/65416 [00:00<?, ?it/s]

Save generated embedding into artifacts for reuse

In [11]:
import os

ARTIFACTS_BASE = '../../../artifacts/step-1-classic-ml/distilbert-base-uncased-finetuned-sst-2-english/'
os.makedirs(ARTIFACTS_BASE, exist_ok=True)

In [12]:
train_df.to_pickle(ARTIFACTS_BASE + 'train_df.pkl')
test_df.to_pickle(ARTIFACTS_BASE + 'test_df.pkl')